In [85]:
import sys, os
import yaml
import json
import requests
import shutil

In [46]:
path = '/api/v4'

In [51]:
conf_file = 'conf/dracoon.yml'
has_conf_file = os.path.isfile(conf_file)
if has_conf_file:
    with open(conf_file, 'r') as stream:
        try:
            conf = yaml.load(stream)
        except yaml.YAMLError as err:
            print(f"ERROR: can't read {conf_file}")
else:
    print(f"ERROR: file {conf_file} doesn't exist")

In [68]:
def login():
    api = '/auth/login'
    url = f"{conf['url']}{path}{api}"
    data = json.dumps({
        'login': conf['user'],
        'password': conf['pass'],
        'authType': 'sql'
    })
    headers = {
        'Content-Type': 'application/json;charset=UTF-8',
        'Accept': 'application/json;charset=UTF-8'
    }
    res = requests.post(url, data=data, headers=headers)
    data = json.loads(res.text)
    return data['token']

token = ''
if 'token' in conf.keys():
    token = conf['token']
else:
    token = login()

In [71]:
def get_sub_nodes(id):
    api = "/nodes"
    query = f"?parent_id={id}"
    url = f"{conf['url']}{path}{api}{query}"
    headers = {'X-Sds-Auth-Token': token}
    res = requests.get(url, headers=headers)
    data = json.loads(res.text)
    return data['items']

In [78]:
def get_node_id(path, opt=None):
    id = 0
    node = None
    path_parts = path.split('/') # remove potentially empty positions
    must_be_file = False # opt['must_be_file'] || false
    
    for path_part in path_parts:
        nodes = get_sub_nodes(id)
        found = False
        for node in nodes:
            if node['name'] == path_part:
                id = node['id']
                found = True
                break
        if not found:
            print("ERROR: Invalid resource path {path}. (File could not be found.)")
    if must_be_file and node["type"] != "file":
        print("ERRPR: Invalid resource path {path}. (Source path points to a folder.)")
    return id

In [92]:
def download(remote, local):
    node_id = get_node_id(remote)
    api = "/nodes/files/#{node_id}/downloads"
    url = f"{conf['url']}{path}{api}"
    headers = {'X-Sds-Auth-Token': token}
    res = requests.get(url, headers=headers, stream=True)
    res.raw.decode_content = True
    with open(local, 'wb') as f:
        shutil.copyfileobj(res.raw, f)

In [93]:
files = [
    '404.jpg',
    'alt_world.jpeg',
    'bad_code.png',
    'fixing_problems.png',
    'geekandpoke_deadline.jpg',
    'jira.jpg',
    'testing.png',
    'The_New_Developer.jpg',
    'toggl-how-to-create-horse-with-programming.jpg']

for file in files:
    download(f"hackaburg/{file}", f"data/{file}")